In [101]:
import experience_study as exp
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from config import db, app

from models import Hive, ExplanatoryVariable
from datetime import datetime

In [102]:
def collect_data(app):
    with app.app_context():
        explanatory_variables = [explanatory_variable.variable_name for explanatory_variable in ExplanatoryVariable.query.filter_by(is_explanatory=True)]
        hives = [hive.to_dict() for hive in Hive.query.all()]

    return explanatory_variables, hives

In [103]:
def rename_ids(hives):
    hives_mod = [{
        **hive,
        'hive_id': hive.pop('id')
    } for hive in hives]

    hives_mod = [{
    **hive,
    'queens': [{
        **queen,
        'queen_id': queen.pop('id')  # Rename 'id' to 'queen_id' using pop
    } for queen in hive['queens']]
    } for hive in hives_mod]

    return hives_mod


In [104]:
def normalize_data(hives_mod):

    metadata_columns = list(hives_mod[0].keys())

    # Remove 'queens' from the metadata columns
    metadata_columns.remove('queens')
    metadata_columns.remove('hive_id')

    df_queens = pd.json_normalize(
        hives_mod, 
        record_path=['queens'],  # Extract 'queens'
        meta=metadata_columns,  
        errors='ignore'
    )

    inspections = df_queens['inspections']
    df_queens.drop(columns=['inspections'], inplace=True)

    df_inspections = pd.json_normalize(
        inspections.explode(),  # Explode to repeat inspections for each row
        errors='ignore'
    )

    df_normalized = pd.merge(df_inspections, df_queens, on='queen_id')
    df_normalized = df.rename(columns={'id': 'inspection_id'})

    return df_normalized

In [105]:
explanatory_variables, hives = collect_data(app)
hives_mod = rename_ids(hives)
df_normalized = normalize_data(hives_mod)

df_normalized.head()

,feeding,capped_brood,fate,inspection_id,humidity,treatment,varroa_mites,egg_count,activity_surrounding_hive,super_count,...,species,date_introduced,status,origin,hive_id,date_added,location_lat,material,user_id,location_long
0,None,Low,Thriving,1,74.0,Other,False,Medium,LOW,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
1,Pollen patty,Medium,Swarmed,2,84.3,Formic acid,True,Low,LOW,1,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
2,Sugar syrup,High,Split,3,57.9,Oxalic acid,True,High,LOW,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
3,Pollen patty,High,Split,4,71.8,Thymol,True,Low,HIGH,3,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
4,Other,High,Swarmed,5,72.6,Thymol,True,Medium,HIGH,2,...,Russian,2024-02-22,clipped,swarm cells,1,2025-01-29,5.601693,Polystyrene,1,78.517182
